## Measurement Threshold of QECCs

### Some functions to 1. generate toric/color code stabiliers, 2. generate random measurement basis, 3. compute entanglement entropy with reference

In [ ]:
# Toric code stabilizer generators

def stabilizers(size):
    N = 2*size**2
    
    stabilizers = []
    
    # Star operators
    for i in range(size**2):
        V = np.zeros((2*(N+2),1))
        V[2*(i//size)*size+i%size][0] = 1
        
        if (i+1)%size==0:
            V[2*(i//size)*size + i%size + 1 - size][0] = 1
            
        else:
            V[2*(i//size)*size + i%size + 1][0] = 1
            
        if (i+1)%size==0:
            V[(2*(i//size)*size + i%size - (size-1) -size)%(2*size**2)][0] = 1
            
        else:
            V[(2*(i//size)*size + i%size - (size-1))%(2*size**2)][0] = 1
            
        if (i+1)%size==0: 
            V[(2*(i//size)*size + i%size +1)%(2*size**2)][0] = 1
            
        else:
            V[(2*(i//size)*size + i%size + size + 1)%(2*size**2)][0] = 1
            
        stabilizers.append(V)
        
    # Plaquette operators
    for i in range(size**2):
        V = np.zeros((2*(N+2),1))
        
        V[N+2 + 2*(i//size)*size+i%size][0] = 1
        V[N+2 + 2*(i//size)*size + i%size + size][0] = 1
        if (i+1)%size==0:
            V[N+2 + 2*(i//size)*size + i%size +1][0] = 1
        else: 
            V[N+2 + 2*(i//size)*size + i%size + size +1][0] = 1
        V[N+2 + (2*(i//size)*size + i%size +2*size)%(2*size**2)][0] = 1
        stabilizers.append(V)
    
    # logical operators
    XX1 = np.zeros((2*(N+2),1))
    for i in range(size):
        XX1[size+i][0] = 1
    XX1[N][0] = 1
    
    XX2 = np.zeros((2*(N+2),1))
    for i in range(size):
        XX2[2*size*i][0] = 1
    XX2[N+1][0] = 1
    
    ZZ2 = np.zeros((2*(N+2),1))
    for i in range(size):
        ZZ2[N+2+i][0] = 1
    ZZ2[2*N+3][0] = 1
        
    ZZ1 = np.zeros((2*(N+2),1))
    
    for i in range(size):
        ZZ1[N+2+ size + i*(2*size)][0] = 1
    ZZ1[2*N+2][0] = 1
    
    stabilizers.append(XX1)
    stabilizers.append(XX2)
    stabilizers.append(ZZ1)
    stabilizers.append(ZZ2)
    
    return stabilizers 


# Check commutation relation

# i = qubit index, v = vector, a = 1, 2, 3 -> Pauli X, Z, Y respectively.
def single_checking(i,a,v): 
    l = len(v)
    if a == 1:
        return v[i + int(l/2) ] == 0 # Z should be checked 
    elif a == 2:
        return v[i] == 0
    else:
        return (v[i] + v[i + int(l/2)])%2 == 0

def checking(u,v):
    a = 0
    l = len(u)
    for i in range(l):
        a =  a + u[i] * v[(i + int(l/2))%l]
    
    return a % 2 == 0 # which means they commute
       
# Stabilizer updatement after measurements
def mul(u,v):
    return (u + v) % 2

def update(i,a,S): # i: qubit index, a: 1,2,3 for Pauli X, Z, Y. 
    c = 0 # check if we removed one stabilizer generator or not
    A = 0 # will store anti-commuting stabilizer
    l = int(len(S[0]))
    M = np.zeros((l,1))
    # stabilizer = S.copy()
    # generating measurement operator
    if a == 1:
        M[i][0] = 1
    elif a == 2:
        M[i + int(l/2)][0] = 1
    else:
        M[i][0] = 1
        M[i + int(l/2)][0] = 1
        
    for k in range(len(S)):
        if c == 0:
            if not single_checking(i,a,S[k]):
                A = S[k]
                S[k] = M
                c = 1
        else:
            if not single_checking(i,a,S[k]):
                S[k] = mul(A,S[k])
     

# Random local projective measurements

def random_measurements(q,p,L,k):
    N = 2*L**2
    S = stabilizers(L)
    set_unmeasure = rand_generate(N,k)
    for i in range(2*L**2):
        if i not in set_unmeasure:  
            r = np.random.random() 
            if r < q:
                update(i,3,S)
            elif r < q + (1-q)*p:
                update(i,1,S)
            else:
                update(i,2,S)
                
    if Entanglement_Entropy(S,L) == 2:
        return True
    else:
        return False

def random_logical(q,p,L):
    N = 2*L**2
    S = stabilizers(L)
    
    for i in range(2*L**2):
        r = np.random.random()
        if r < q:
            update(i,3,S)
        elif r < q + (1-q)*p:
            update(i,1,S)
        else:
            update(i,2,S)
    
    return measured_logical(S,L)        

# Generating a random set of unmeasuring qubits
def rand_generate(N,p):
    n = round(N*p)
    set = []
    while len(set) < n:
        i = np.random.randint(0,N-1)
        if i not in set:
            set.append(i)
    return set

# The idea is to count the number of anti-commuting pair after projecting operators into the reference.
# Compute Entanglement Entropy of given updated stabilizer state
def Entanglement_Entropy(S,L):
    
    # We now project stabilizer onto the region R
    l = len(S)
    S_R = np.zeros((l,4))
    for i in range(l):
        S_R[i][0] = S[i][2*L**2]
        S_R[i][1] = S[i][2*L**2+1]
        S_R[i][2] = S[i][4*L**2+2]
        S_R[i][3] = S[i][4*L**2+3]
    EE = np.linalg.matrix_rank(S_R) - 2
    
    return EE

# Check measured logical operator
def measured_logical(S,L):
    # collect the Pauli operators consisting stabilizer of reference
    gen = [0,0,0,0,0,0] # X1, X2, Z1, Z2, Y1, Y2
    measured_operator = [] # [X,Y,Z,entangled]
    l = len(S)
    
    for i in range(l):
        if S[i][4*L**2+2] == 1:
            if S[i][2*L**2] == 1:
                gen[4] = 1
            else:
                gen[2] = 1
        elif S[i][2*L**2] ==1:
            gen[0] = 1
        
        if S[i][4*L**2+3] == 1:
            if S[i][2*L**2+1] == 1:
                gen[5] = 1
            else:
                gen[3] = 1
        elif S[i][2*L**2+1] ==1:
            gen[1] = 1
    
    if (1-gen[1]*gen[3])*(1-gen[1]*gen[5])*(1-gen[3]*gen[5]) == 0: # they are stabilized by XX, ZZ, YY
        if (1-gen[0]*gen[2])*(1-gen[0]*gen[4])*(1-gen[2]*gen[4]) == 0:
            measured_operator.append([0,0,0,1])
            measured_operator.append([0,0,0,1])
        
    else:
        if gen[0] == 1:
            measured_operator.append([1,0,0,0])
        if gen[2] == 1:
                measured_operator.append([0,0,1,0])
        if gen[4] == 1:
            measured_operator.append([0,1,0,0])
    
        if gen[1] == 1:
            measured_operator.append([1,0,0,0])
        if gen[3] == 1:
            measured_operator.append([0,0,1,0])
        if gen[5] == 1:
            measured_operator.append([0,1,0,0])
    
    return measured_operator

########################################################################################
# From this line, quantum color code simulation starts. 
# Every function related to color code is defined in _color 

# Generating color code
def stabilizer_color(L): # L starts from 1
    plaq_coord = [] # stabilizer in (a,b) coordinate notation
    
    def end(a):
        k = (a+1)//3
        r = (a+1)%3       
        if r == 0:
            return 2*k-1
        else:
            return 2*k
    
    def hexagonal(a,b):
        r = a%3
        if r == 2:
            return [[a,b], [a,b+1], [a+1,b], [a+1,b+1], [a+2,b], [a+2,b+1]]
        elif r == 0:
            return [[a,b], [a,b+1], [a+1,b], [a+1,b+1], [a+2,b+1], [a+2,b+2]]
        else:
            return [[a,b], [a,b+1], [a+1,b+1], [a+1,b+2], [a+2,b+1], [a+2,b+2] ]
    
    for i in range(L): # boundary plaquette
        plaq_coord.append([[3*i,0],[3*i+1,0],[3*i+2,0],[3*i+2,1]]) # left boundary
        plaq_coord.append([[1+3*i, end(1+3*i)], [2+3*i, end(2+3*i)], [3+3*i, end(3+3*i)-1],[3+3*i, end(3+3*i)]])
        plaq_coord.append([[3*L,2*i],[3*L,2*i+1],[3*L-1,2*i],[3*L-1,2*i+1]]) # bottom boundary
        
    for i in range(L-1): # Hexagonal plaquette
        for j in range(i+1):
            plaq_coord.append(hexagonal(2+3*i,2*j))
            plaq_coord.append(hexagonal(2+3*i+1,2*j+1))
            plaq_coord.append(hexagonal(2+3*i+2,2*j))
    
    # Converting coordinate to physical index
    
    def convert(s):
        phy_index = []
        for i in range(len(s)):
            k = s[i][0]//3
            r = s[i][0]%3
            b = s[i][1]
            phy_index.append(3*k**2+k+(2*k+1)*r+b)
        
        return phy_index 
    
    plaq = []
    
    for i in range(len(plaq_coord)):
        plaq.append(convert(plaq_coord[i]))
    
    # Converting physical index into vector
    
    N_phy = 1 + 3*L*(L+1)
    stabilizers = []
    
    for s in plaq:
        V1 = np.zeros((2*(N_phy+1),1))
        V2 = np.zeros((2*(N_phy+1),1))
        for i in s:
            V1[i][0] = 1 # X plaquette
            V2[N_phy+1 + i][0] = 1 # Z plaquette
        
        stabilizers.append(V1)
        stabilizers.append(V2)
    
    # Adding entangled logical operator
    Logical_coord = []
    
    Logical_coord.append([3,0]); Logical_coord.append([4,0]); Logical_coord.append([6,2]); 
    Logical_coord.append([6,3]); Logical_coord.append([6,4]); # left and right condensation
    
    for i in range(L-2):
        Logical_coord.append([8+3*i,2])
        Logical_coord.append([9+3*i,2])
    
    Logical = convert(Logical_coord)
    
    # Converting logical operator coordinate into physical qubit index
    
    # insert in the stabilizer
    VX = np.zeros((2*(N_phy+1),1))
    VZ = np.zeros((2*(N_phy+1),1))
    
    for i in Logical:
        VX[i][0] = 1
        VZ[N_phy + 1 + i][0] = 1
    
    VX[N_phy][0] = 1 
    VZ[2*N_phy+1][0] = 1
    
    stabilizers.append(VX)
    stabilizers.append(VZ)
    
    return stabilizers

def Entanglement_Entropy_color(S,L):
    N_phy = 1 + 3*L*(L+1)
    
    # We now project stabilizer onto the region R
    l = len(S)
    S_R = np.zeros((l,2))
    for i in range(l):
        S_R[i][0] = S[i][N_phy]
        S_R[i][1] = S[i][2*N_phy+1]
    EE = np.linalg.matrix_rank(S_R) - 1
    
    return EE


def random_measurements_color(q,p,L,k):
    N_phy = 1 + 3*L*(L+1)
    S = stabilizer_color(L)
    set_unmeasure = rand_generate(N_phy,k)
    
    for i in range(N_phy):
        if i not in set_unmeasure:  
            r = np.random.random() 
            if r < q:
                update(i,3,S)
            elif r < q + (1-q)*p:
                update(i,1,S)
            else:
                update(i,2,S)
                
    if Entanglement_Entropy_color(S,L) == 1:
        return True
    else:
        return False


def measured_logical_color(S,L):
    N_phy = 1 + 3*L*(L+1)
    
    for i in range(len(S)):
        
        if S[i][N_phy] == 1:
            if S[i][2*N_phy+1] == 1:
                return [0,1,0] # Y operator
            else:
                return [1,0,0] # X operator
        
        if S[i][2*N_phy+1] == 1:
            return [0,0,1] # Z operator


def random_logical_color(p,q,L):
    N_phy = 1 + 3*L*(L+1)
    S = stabilizer_color(L)
    
    for i in range(N_phy):
        r = np.random.random()
        if r < p:
            update(i,3,S)
        elif r < p + (1-p)*q:
            update(i,1,S)
        else:
            update(i,2,S)
    
    return measured_logical_color(S,L) 

### Toric code simulation for 1) Measurement threshold. 2) Measured logical operator distribution 

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import functions
import multiprocessing as mp
import csv
#import tqdm

# This is a function measuring info preservation probability of toric code    
def success_prob(L,rep,p,q,percent): 
    # L: lattice size, rep: number of repetition, p: Y bais probability, q: X bais probability , percent: rate of unmeasured qubits
    
    count = []
    
    def check(r):
        if r: count.append(1)   
        
    pool = mp.Pool(80)
    
    for n in range(rep):
        pool.apply_async(functions.random_measurements,(p,q,L,percent),callback=check)
    pool.close()
    pool.join()
    
    return len(count)/rep


# This is a function compute distribution of measured logical operators of toric code.
def operator_distribution(p,q,L,rep):
    set1 = []
    set2 = []
    
    def addition(M):
        set1.append(M[0]) 
        set2.append(M[1])
    
    pool = mp.Pool(80)
    
    for n in range(rep):
        pool.apply_async(functions.random_logical,(p,q,L),callback=addition)
    pool.close()
    pool.join()
    
    otr_dist1 = np.array([0,0,0,0])
    otr_dist2 = np.array([0,0,0,0])
    
    for i in range(len(set1)):
        otr_dist1 = otr_dist1 + set1[i]
        otr_dist2 = otr_dist2 + set2[i]
        
    return [otr_dist1/rep, otr_dist2/rep]




### Color code simulation for 1) Measurement threshold. 2) Measured logical operator distribution 

In [ ]:
# This is a function measuring info preservation probability of color code
def success_prob_color(L,rep,p,q,percent):
   
    count = []
    
    def check(r):
        if r: count.append(1)   
        
    pool = mp.Pool(80)
    
    for n in range(rep):
        pool.apply_async(functions.random_measurements_color,(p,q,L,percent),callback=check)
    pool.close()
    pool.join()
    
    return len(count)/rep

# This is a function compute distribution of measured logical operators of color code.
def operator_distribution_color(p,q,L,rep):
    set = []
    
    def addition(M):
        print(M)
        set.append(M)
            
    pool = mp.Pool(80)
    
    set = pool.starmap(functions.random_logical_color, [(p,q,L) for i in range(rep)])
    print(set)
    pool.close()
    pool.join()
    
    opr_dist = np.array([0,0,0])
    
    for i in range(len(set)):
        opr_dist = opr_dist + set[i]
    
    return opr_dist/rep